---
# Creating and Persisting DataFrames
---

In [1]:
import numpy as np
import pandas as pd

Create parallel lists with data in them. Each of these lists will be a column in the
DataFrame, so they should have the same type

In [2]:
fname = ["Paul", "John", "Richard", "George"]
lname = ["McCartney", "Lennon", "Starkey", "Harrison"]
birth = [1942, 1940, 1940, 1943]

Create a dictionary from the lists, mapping the column name to the list:

In [3]:
people = dict(first=fname, last=lname, birth=birth)

Create a DataFrame from the dictionary

In [4]:
beatles = pd.DataFrame(people)

In [5]:
beatles

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [6]:
beatles.index

RangeIndex(start=0, stop=4, step=1)

In [11]:
# change index
pd.DataFrame(data=people, index=list('abcd'))

,first,last,birth
a,Paul,McCartney,1942
b,John,Lennon,1940
c,Richard,Starkey,1940
d,George,Harrison,1943


## Writing CSV

Write the DataFrame to a CSV file:

In [12]:
beatles

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [13]:
 from io import StringIO

In [21]:
beatles_file = StringIO()
beatles.to_csv(beatles_file)

Look at the file contents:

In [22]:
print(beatles_file.getvalue())

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943



In [24]:
_ = beatles_file.seek(0)
pd.read_csv(beatles_file)

,Unnamed: 0,first,last,birth
0,0,Paul,McCartney,1942
1,1,John,Lennon,1940
2,2,Richard,Starkey,1940
3,3,George,Harrison,1943


The `read_csv` function has an `index_col` parameter that you can use to specify the
location of the index:

In [25]:
_ = beatles_file.seek(0)
pd.read_csv(beatles_file, index_col=0)

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


Alternatively, if we didn't want to include the index when writing the CSV file, we can set the
index parameter to `False`:

In [26]:
beatles_file = StringIO()
beatles.to_csv(beatles_file, index=False)

In [28]:
_ = beatles_file.seek(0)
pd.read_csv(beatles_file)

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


## Reading large CSV files

The pandas library is an in-memory tool. You need to be able to fit your data in memory to use pandas with it. If you come across a large CSV file that you want to process, you have a few options. If you can process portions of it at a time, you can read it into chunks and process each chunk. Alternatively, if you know that you should have enough memory to load the file, there are a few hints to help pare down the file size.  
Note that in general, you should have three to ten times the amount of memory as the size of the DataFrame that you want to manipulate.  Extra memory should give you enough extra space to perform many of the common operations. 

In [29]:
diamonds = pd.read_csv('./diamonds.csv', nrows=1000)
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


`.info` method to see how much memory the sample of data uses

In [30]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    1000 non-null   float64
 1   cut      1000 non-null   object 
 2   color    1000 non-null   object 
 3   clarity  1000 non-null   object 
 4   depth    1000 non-null   float64
 5   table    1000 non-null   float64
 6   price    1000 non-null   int64  
 7   x        1000 non-null   float64
 8   y        1000 non-null   float64
 9   z        1000 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 78.2+ KB


Use the `dtype` parameter to `read_csv` to tell it to use the correct (or smaller) numeric types

In [31]:
diamonds2 = pd.read_csv('./diamonds.csv', nrows=1000, dtype={
    'carat': np.float32,
    'depth': np.float32,
    'table': np.float32,
    "x": np.float32,
    "y": np.float32,
    "z": np.float32,
    "price": np.int16,
})

diamonds2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    1000 non-null   float32
 1   cut      1000 non-null   object 
 2   color    1000 non-null   object 
 3   clarity  1000 non-null   object 
 4   depth    1000 non-null   float32
 5   table    1000 non-null   float32
 6   price    1000 non-null   int16  
 7   x        1000 non-null   float32
 8   y        1000 non-null   float32
 9   z        1000 non-null   float32
dtypes: float32(6), int16(1), object(3)
memory usage: 49.0+ KB


Make sure that summary statistics are similar with our new dataset to the original


In [33]:
diamonds.describe().equals(diamonds2.describe())

False

In [34]:
diamonds.describe().T

,count,mean,std,min,25%,50%,75%,max
carat,1000.0,0.68928,0.195291,0.20,0.70,0.71,0.79,1.27
depth,1000.0,61.72280,1.758879,53.00,60.90,61.80,62.60,69.50
table,1000.0,57.73470,2.467946,52.00,56.00,57.00,59.00,70.00
price,1000.0,2476.54000,839.575620,326.00,2777.00,2818.00,2856.00,2898.00
x,1000.0,5.60594,0.625173,3.79,5.64,5.77,5.92,7.12
y,1000.0,5.59918,0.611974,3.75,5.63,5.76,5.91,7.05
z,1000.0,3.45753,0.389819,2.27,3.45,3.55,3.64,4.33


In [35]:
diamonds2.describe().T

,count,mean,std,min,25%,50%,75%,max
carat,1000.0,0.689281,0.195291,0.20,0.700000,0.710000,0.790000,1.27
depth,1000.0,61.722824,1.758878,53.00,60.900002,61.799999,62.599998,69.50
table,1000.0,57.734699,2.467944,52.00,56.000000,57.000000,59.000000,70.00
price,1000.0,2476.540000,839.575620,326.00,2777.000000,2818.000000,2856.000000,2898.00
x,1000.0,5.605941,0.625173,3.79,5.640000,5.770000,5.920000,7.12
y,1000.0,5.599180,0.611972,3.75,5.630000,5.760000,5.910000,7.05
z,1000.0,3.457533,0.389819,2.27,3.450000,3.550000,3.640000,4.33


Use the `dtype` parameter to use change object types to categoricals. First, inspect the `.value_counts` method of the object columns. If they are low cardinality, you can convert them to categorical columns to save even more memory

In [40]:
diamonds2.select_dtypes(include='object').columns

Index(['cut', 'color', 'clarity'], dtype='object')

In [41]:
diamonds2.cut.value_counts()

Ideal        333
Premium      290
Very Good    226
Good          89
Fair          62
Name: cut, dtype: int64

In [42]:
diamonds2.color.value_counts()

E    240
F    226
G    139
D    129
H    125
I     95
J     46
Name: color, dtype: int64

In [43]:
diamonds2.clarity.value_counts()

SI1     306
VS2     218
VS1     159
SI2     154
VVS2     62
VVS1     58
I1       29
IF       14
Name: clarity, dtype: int64

Because these are of low cardinality, we can convert them to categoricals and use
around 37% of the original size

In [44]:
diamonds3 = pd.read_csv('./diamonds.csv', nrows=1000,
                        dtype={
                                'carat': np.float32,
                                'depth': np.float32,
                                'table': np.float32,
                                "x": np.float32,
                                "y": np.float32,
                                "z": np.float32,
                                "price": np.int16,
                               "cut": "category",
                               "color": "category",
                               "clarity": "category",
                              },
                        )
diamonds3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    1000 non-null   float32 
 1   cut      1000 non-null   category
 2   color    1000 non-null   category
 3   clarity  1000 non-null   category
 4   depth    1000 non-null   float32 
 5   table    1000 non-null   float32 
 6   price    1000 non-null   int16   
 7   x        1000 non-null   float32 
 8   y        1000 non-null   float32 
 9   z        1000 non-null   float32 
dtypes: category(3), float32(6), int16(1)
memory usage: 29.4 KB


If there are columns that we know we can ignore, we can use the usecols
parameter to specify the columns we want to load. Here, we will ignore columns x, y,
and z:

In [45]:
cols = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price']
diamonds4 = pd.read_csv('./diamonds.csv', nrows=1000,
                        dtype={
                                'carat': np.float32,
                                'depth': np.float32,
                                'table': np.float32,
                                "x": np.float32,
                                "y": np.float32,
                                "z": np.float32,
                                "price": np.int16,
                               "cut": "category",
                               "color": "category",
                               "clarity": "category",
                              },
                        usecols=cols,
                        )
diamonds4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    1000 non-null   float32 
 1   cut      1000 non-null   category
 2   color    1000 non-null   category
 3   clarity  1000 non-null   category
 4   depth    1000 non-null   float32 
 5   table    1000 non-null   float32 
 6   price    1000 non-null   int16   
dtypes: category(3), float32(3), int16(1)
memory usage: 17.7 KB


If the preceding steps are not sufficient to create a small enough DataFrame, you might still be in luck. If you can process chunks of the data at a time and do not need all of it in memory, you can use the `chunksize` parameter

In [46]:
diamonds_iter = pd.read_csv('./diamonds.csv', nrows=1000,
                        dtype={
                                'carat': np.float32,
                                'depth': np.float32,
                                'table': np.float32,
                                "x": np.float32,
                                "y": np.float32,
                                "z": np.float32,
                                "price": np.int16,
                               "cut": "category",
                               "color": "category",
                               "clarity": "category",
                              },
                        usecols=cols,
                        chunksize=200,
                        )

In [47]:
def process(df):
  return (
      f"proceed {df.size} items"
  )

In [50]:
for chunk in diamonds_iter:
  process(chunk)

In [51]:
np.iinfo(np.int8)

iinfo(min=-128, max=127, dtype=int8)

In [52]:
np.finfo(np.float16)

finfo(resolution=0.001, min=-6.55040e+04, max=6.55040e+04, dtype=float16)

In [53]:
diamonds.price.memory_usage()

8128

In [54]:
diamonds.price.memory_usage(index=False)

8000

In [55]:
diamonds.cut.memory_usage()

8128

In [56]:
diamonds.cut.memory_usage(deep=True)

63461

In [57]:
diamonds4.to_feather('/tmp/d.arr')

In [58]:
diamonds5 = pd.read_feather('/tmp/d.arr')

In [59]:
diamonds4.to_parquet('/tmp/d.pqt')

In [60]:
diamonds4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    1000 non-null   float32 
 1   cut      1000 non-null   category
 2   color    1000 non-null   category
 3   clarity  1000 non-null   category
 4   depth    1000 non-null   float32 
 5   table    1000 non-null   float32 
 6   price    1000 non-null   int16   
dtypes: category(3), float32(3), int16(1)
memory usage: 17.7 KB
